In [ ]:
!pip install -U scikit-fuzzy
import numpy as np
import skfuzzy as fuzz
from skfuzzy import control as ctrl

In [ ]:

temp= ctrl.Antecedent(np.arange(0, 41), 'temperature')
hum = ctrl.Antecedent(np.arange(0, 101), 'humidity')


cmd = ctrl.Consequent(np.arange(15, 27), 'command')

In [ ]:

temp['coldest'] = fuzz.trapmf(temp.universe, [0, 4, 6, 8])
temp['cold'] = fuzz.trapmf(temp.universe, [6, 10, 12, 16])
temp['warm'] = fuzz.trapmf(temp.universe, [12, 16, 18, 24])
temp['hot'] = fuzz.trapmf(temp.universe, [18, 22, 24, 32])
temp['hottest'] = fuzz.trapmf(temp.universe, [24, 28, 30, 40])


hum['low'] = fuzz.gaussmf(hum.universe, 0, 30) # 0,15
hum['optimal'] = fuzz.gaussmf(hum.universe, 50, 15) # 50, 15
hum['high'] = fuzz.gaussmf(hum.universe, 100, 50) # 100,15


cmd['cool'] = fuzz.trimf(cmd.universe, [15, 17, 20])
cmd['warmup'] = fuzz.trimf(cmd.universe, [18, 20, 26])
# Rule system
# Rules for warming up
rule1 = ctrl.Rule(
    (temp['coldest'] & hum['low']) |
    (temp['coldest'] & hum['optimal']) |
    (temp['coldest'] & hum['high']) |
    (temp['cold'] & hum['low']) |
    (temp['cold'] & hum['optimal']) |
    (temp['warm'] & hum['low']), cmd['warmup'])

# Rules for cooling up
rule2 = ctrl.Rule(
    (temp['warm'] & hum['optimal']) |
    (temp['warm'] & hum['high']) |
    (temp['hot'] & hum['optimal']) |
    (temp['hot'] & hum['high']) |
    (temp['hottest'] & hum['low']) |
    (temp['hottest'] & hum['optimal']) |
    (temp['hottest'] & hum['high']), cmd['cool'])


cmd_ctrl = ctrl.ControlSystem([rule1, rule2])
cmd_output = ctrl.ControlSystemSimulation(cmd_ctrl)

In [ ]:
def generateOutput(temperature_value, humidity_value):
    cmd_output.input['temperature'] = temperature_value
    cmd_output.input['humidity'] = humidity_value

    cmd_output.compute()

    print("Command is defined between 15 y 26")
    return cmd_output.output['command']
    # re_temp = round(cmd_output.output['command'], 1)
    # if (cmd_output.output['command'] > 20):
    #     return 'Warm up', re_temp
    # elif (cmd_output.output['command'] < 20 and cmd_output.output['command'] > 18):
    #     return 'No change', re_temp
    # else:
    #     return 'Cool Up', re_temp

In [ ]:
temperature = float(input("Current Temperature:"))
humidity = float(input("Current Humidity:"))
out = generateOutput(temperature, humidity)
print("Crisp value : ", out)
print("The AC should Set itself to ", round(out), " acccording to the rules given")

Current Temperature:35
Current Humidity:90
Command is defined between 15 y 26
Crisp value :  17.38888888888889
The AC should Set itself to  17  acccording to the rules given
